# OA Gold from Silver
Create Gold tables as copies of Silver normalised tables, WITHOUT audit columns and WITH consistent versioning columns.

**Silver Sources**:
- `silver_md.md_dta_operational_agreement_draft`
- `silver_md.md_dta_oa_attributes_draft`
- `silver_md.md_dta_oa_options_draft`
- `silver_md.md_dta_oa_other_draft`

**Gold Targets**:
- `gold_md.md_dta_operational_agreement`
- `gold_md.md_dta_oa_attributes`
- `gold_md.md_dta_oa_options`
- `gold_md.md_dta_oa_other`


In [ ]:
# Cell 0: Imports

import json
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, TimestampType

from clinical_data_standards_framework.utils import compute_definition_hash

print("=" * 80)
print("OA GOLD FROM SILVER")
print("=" * 80)


In [ ]:
# Cell 1: Read configuration from setup task

globals_dict = json.loads(
    dbutils.jobs.taskValues.get(taskKey="setup", key="globals")
)

created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id    = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name  = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id    = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

catalog       = globals_dict["catalog"]
silver_schema = globals_dict["silver_schema"]
gold_schema   = globals_dict["gold_schema"]

print("Resolved globals:")
print(f"  catalog        = {catalog}")
print(f"  silver_schema  = {silver_schema}")
print(f"  gold_schema    = {gold_schema}")

spark.sql(f"USE CATALOG `{catalog}`")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{gold_schema}`")


In [ ]:
# Cell 2: Define audit columns to drop and table mappings

AUDIT_COLS = [
    "created_by_principal",
    "created_timestamp",
    "created_at",
    "created_on",
    "updated_by_principal",
    "updated_timestamp",
    "updated_at",
    "updated_on",
    "databricks_job_id",
    "databricks_job_name",
    "databricks_run_id",
    "inserted_timestamp",
    "last_updated_timestamp",
    "audit_insert_ts",
    "audit_update_ts",
    "created_ts",
    "last_updated_ts",
]

# Silver → Gold table mappings
table_mappings = [
    # (silver_table_name, gold_table_name)
    ("md_dta_operational_agreement_draft", "md_dta_operational_agreement"),
    ("md_dta_oa_attributes_draft",         "md_dta_oa_attributes"),
    ("md_dta_oa_options_draft",            "md_dta_oa_options"),
    ("md_dta_oa_other_draft",              "md_dta_oa_other"),
]

print(f"Table mappings configured: {len(table_mappings)} tables")


In [ ]:
# Cell 3: Helper UDF for definition hash

@F.udf(StringType())
def oa_header_definition_hash_udf(
    operational_agreement_id: str,
    trial_id: str,
    data_stream: str,
    data_provider_name: str
) -> str:
    record = {
        "operational_agreement_id": operational_agreement_id or "",
        "trial_id": trial_id or "",
        "data_stream": data_stream or "",
        "data_provider_name": data_provider_name or "",
    }
    return compute_definition_hash(record, list(record.keys()))

print("✅ Definition hash UDF defined")


In [ ]:
# Cell 4: Helper to add standard versioning columns

def add_standard_gold_version_columns(df, gold_name: str):
    """
    Adds / normalises the standard versioning columns for Gold OA tables:
      - library_version
      - is_major_version
      - is_dta_major
      - parent_version
      - effective_start_ts
      - effective_end_ts
      - is_current
    """

    # Derive library_version from version_tag where possible
    if "library_version" not in df.columns:
        if "version_tag" in df.columns:
            df = df.withColumn(
                "library_version",
                F.when(F.col("version_tag").isNotNull(),
                       F.split(F.col("version_tag"), "-").getItem(0))
                 .otherwise(F.lit(None).cast("string"))
            )
        else:
            df = df.withColumn("library_version", F.lit(None).cast("string"))

    if "is_major_version" not in df.columns:
        df = df.withColumn("is_major_version", F.lit(True))

    if "is_dta_major" not in df.columns:
        df = df.withColumn("is_dta_major", F.lit(True))

    if "parent_version" not in df.columns:
        df = df.withColumn("parent_version", F.lit(None).cast("string"))

    if "effective_start_ts" not in df.columns:
        df = df.withColumn("effective_start_ts", F.current_timestamp().cast(TimestampType()))

    if "effective_end_ts" not in df.columns:
        df = df.withColumn("effective_end_ts", F.lit(None).cast(TimestampType()))

    if "is_current" not in df.columns:
        df = df.withColumn("is_current", F.lit(True))

    return df

print("✅ Standard Gold version columns helper defined")


In [ ]:
# Cell 5: Helper to create/overwrite Gold table from Silver

def build_gold_from_silver(silver_name: str, gold_name: str):
    silver_full = f"{catalog}.{silver_schema}.{silver_name}"
    gold_full   = f"{catalog}.{gold_schema}.{gold_name}"

    print("\n" + "=" * 80)
    print(f"Processing Silver → Gold")
    print(f"  Silver: {silver_full}")
    print(f"  Gold  : {gold_full}")
    print("=" * 80)

    if not spark.catalog.tableExists(silver_full):
        print(f"⚠ Skipping: Silver table does not exist: {silver_full}")
        return 0

    df_silver = spark.table(silver_full)

    # Drop audit columns
    existing_audit_cols = [c for c in AUDIT_COLS if c in df_silver.columns]
    if existing_audit_cols:
        print(f"Dropping audit columns from Silver: {existing_audit_cols}")
        df_gold = df_silver.drop(*existing_audit_cols)
    else:
        print("No audit columns found to drop.")
        df_gold = df_silver

    # For md_dta_operational_agreement GOLD, ensure OA-specific fields are present
    if gold_name == "md_dta_operational_agreement":
        # definition_hash
        if "definition_hash" not in df_gold.columns:
            df_gold = df_gold.withColumn(
                "definition_hash",
                oa_header_definition_hash_udf(
                    F.col("operational_agreement_id"),
                    F.col("trial_id"),
                    F.col("data_stream"),
                    F.col("data_provider_name"),
                )
            )

        # row_status from any existing status-style column
        if "row_status" not in df_gold.columns:
            if "status" in df_gold.columns:
                df_gold = df_gold.withColumn("row_status", F.col("status"))
            else:
                df_gold = df_gold.withColumn("row_status", F.lit("COMPLETED"))

        # vendor_comments – use vendor_comments if available
        if "vendor_comments" not in df_gold.columns:
            df_gold = df_gold.withColumn("vendor_comments", F.lit(None).cast("string"))

    # Add / normalise the standard Gold version columns for ALL OA gold tables
    df_gold = add_standard_gold_version_columns(df_gold, gold_name)

    row_count = df_gold.count()
    print(f"Rows to write to Gold: {row_count}")

    if row_count == 0:
        print(f"⚠ No data in Silver table {silver_full}. Gold will be created empty.")

    (
        df_gold
        .write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(gold_full)
    )

    print(f"✅ Wrote {row_count} rows to {gold_full}")
    return row_count

print("✅ Gold builder function defined")


In [ ]:
# Cell 6: Run for all 4 tables

total_written = {}

for silver_name, gold_name in table_mappings:
    count = build_gold_from_silver(silver_name, gold_name)
    total_written[gold_name] = count

print("\nSummary of Gold table writes:")
for gold_name, cnt in total_written.items():
    print(f"  {gold_name}: {cnt} rows")

# Expose a task value for downstream jobs
dbutils.jobs.taskValues.set(
    key="oa_gold_build_summary",
    value=json.dumps(total_written)
)

print("\n✅ Gold build completed.")
